In [39]:
from numpy import *
dataset=[]
n=int(input("Enter number of transactions:"))
for i in range (n):
    print('Enter comma separated transaction:')
    line=input()
    dataset.append(line.split(','))
print(dataset)    

Enter number of transactions:4
Enter comma separated transaction:
1,3,4
Enter comma separated transaction:
2,3,5
Enter comma separated transaction:
1,2,3,5
Enter comma separated transaction:
2,5
[['1', '3', '4'], ['2', '3', '5'], ['1', '2', '3', '5'], ['2', '5']]


In [40]:
def createC1(dataset):
    C1=[]
    for transaction in dataset:
        for item in transaction:
            if not [item] in C1:
                C1.append([item])
    C1.sort()
    return list(map(frozenset,C1))

def scanD(D,Ck,minSupport):
    ssCnt={}
    for tid in D:
        for can in Ck:
            if can.issubset(tid):
                if not can in ssCnt:ssCnt[can]=1
                else:ssCnt[can]+=1
    numItems=float(len(D))
    retList=[]
    supportData={}
    for key in ssCnt:
        support=ssCnt[key]#removed division by numItems
        if support>=minSupport:
            retList.insert(0,key)
        supportData[key]=support
    return retList,supportData

In [42]:
D=list(map(set,dataset))#make dataset from list to set
D

[{'1', '3', '4'}, {'2', '3', '5'}, {'1', '2', '3', '5'}, {'2', '5'}]

In [41]:
C1=createC1(dataset)
C1

[frozenset({'1'}),
 frozenset({'2'}),
 frozenset({'3'}),
 frozenset({'4'}),
 frozenset({'5'})]

In [43]:
L1,suppDat0 = scanD(D,C1,2)
L1

[frozenset({'5'}), frozenset({'2'}), frozenset({'3'}), frozenset({'1'})]

In [44]:
def aprioriGen(Lk,k):
    retList=[]
    for i in range(len(Lk)):
        for j in range(i+1,len(Lk)):
            L1=list(Lk[i])[:k-2]
            L2=list(Lk[j])[:k-2]
            L1.sort()
            L2.sort()
            if L1==L2:
                retList.append(Lk[i] | Lk[j])
    return retList

In [45]:
def apriori(dataSet,minSupport):
    C1=createC1(dataset)
    D=list(map(set,dataset))
    L1,supportData=scanD(D,C1,minSupport)
    L=[L1]
    k=2
    while(len(L[k-2])>0):
        Ck=aprioriGen(L[k-2],k)
        Lk,supK = scanD(D,Ck,minSupport)#Scan DB to get Lk
        supportData.update(supK)
        L.append(Lk)
        k+=1
    return L,supportData

In [46]:
L,suppData=apriori(dataset,2)
L#first part of answer got, 

[[frozenset({'5'}), frozenset({'2'}), frozenset({'3'}), frozenset({'1'})],
 [frozenset({'2', '3'}),
  frozenset({'3', '5'}),
  frozenset({'2', '5'}),
  frozenset({'1', '3'})],
 [frozenset({'2', '3', '5'})],
 []]

In [ ]:
aprioriGen(L[0],2)#using aprioriGen, basically forms combinations

# Now to generate rules and calculate confidence

In [48]:
def calcConf(freqSet,H,supportData,brl,minConf):
    prunedH=[]
    for conseq in H:
        conf=supportData[freqSet]/supportData[freqSet - conseq]
        if conf >= minConf:
            print(freqSet-conseq,'-->',conseq,'conf:',conf)
            brl.append((freqSet-conseq,conseq,conf))
            prunedH.append(conseq)
    return prunedH

In [49]:
def rulesFromConseq(freqSet,H,supportData,brl,minConf):
    m=len(H[0])
    if(len(freqSet) > (m+1)):
        Hmp1 = aprioriGen(H,m+1)
        Hmp1 = calcConf(freqSet,Hmp1,supportData,brl,minConf)
        if (len(Hmp1) > 1):
            rulesFromConseq(freqSet,Hmp1,supportData,brl,minConf)

In [66]:
def generateRules(L,supportData,minConf):
    bigRuleList=[]
    for i in range(1,len(L)):
        print('i=',i)
        for freqSet in L[i]:
            print('freqSet=',freqSet)
            H1=[frozenset([item]) for item in freqSet]
            if(i > 1):
                rulesFromConseq(freqSet,H1,supportData,bigRuleList,minConf)
            else:
                calcConf(freqSet,H1,supportData,bigRuleList,minConf)
    return bigRuleList
            

In [68]:
L,suppData = apriori(dataset,2)
rules=generateRules(L,suppData,minConf=0.7)
print('rules=\n',rules)

i= 1
freqSet= frozenset({'2', '3'})
freqSet= frozenset({'3', '5'})
freqSet= frozenset({'2', '5'})
frozenset({'5'}) --> frozenset({'2'}) conf: 1.0
frozenset({'2'}) --> frozenset({'5'}) conf: 1.0
freqSet= frozenset({'3', '1'})
frozenset({'1'}) --> frozenset({'3'}) conf: 1.0
i= 2
freqSet= frozenset({'2', '3', '5'})
i= 3
rules=
 [(frozenset({'5'}), frozenset({'2'}), 1.0), (frozenset({'2'}), frozenset({'5'}), 1.0), (frozenset({'1'}), frozenset({'3'}), 1.0)]
